In [1]:
import warnings
import numpy as np
import pandas as pd
import pickle
import time
import pymongo
# import matplotlib.pyplot as plt
# import plotly.graph_objs as go
# from plotly.offline import init_notebook_mode, iplot
# from sklearn.metrics import mean_squared_error
# from sklearn.model_selection import train_test_split

from keras import optimizers
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
# %matplotlib inline
warnings.filterwarnings("ignore")

# Set seeds to make the experiment more reproducible.
# from tensorflow import set_random_seed
from numpy.random import seed
# set_random_seed(1)
seed(1)

Using TensorFlow backend.
/home/edwin/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/edwin/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/edwin/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/edwin/.local/lib/python3.7/site-packages/tensorflow/python/fra

In [2]:
train_df = pd.read_csv('./test/train2.csv', parse_dates=['date'])
test_df = pd.read_csv('./test/test.csv', parse_dates=['date'])

In [3]:
def split_data(train_data,test_data):
    train_data['date'] = pd.to_datetime(train_data['date'])
    test_data['date'] = pd.to_datetime(test_data['date'])

    train_data['month'] = train_data['date'].dt.month
    train_data['day'] = train_data['date'].dt.dayofweek
    train_data['year'] = train_data['date'].dt.year

    test_data['month'] = test_data['date'].dt.month
    test_data['day'] = test_data['date'].dt.dayofweek
    test_data['year'] = test_data['date'].dt.year

    col = [i for i in test_data.columns if i not in ['date','id']]
    y = 'sales'
    train_x, test_x, train_y, test_y = train_test_split(train_data[col],train_data[y], test_size=0.2, random_state=2018)
    return (train_x, test_x, train_y, test_y,col)

train_x, test_x, train_y, test_y,col = split_data(train_df,test_df)

In [4]:
train_x.shape, test_x.shape

((730400, 5), (182600, 5))

In [5]:
# reshape input to be [samples, time steps, features]
train_x = np.array(train_x).reshape(train_x.shape[0], 1, train_x.shape[1])
test_x = np.array(test_x).reshape(test_x.shape[0], 1, test_x.shape[1])
train_x.shape,test_x.shape

((730400, 1, 5), (182600, 1, 5))

In [6]:
_optimiser = ['Adam','Nadam','RMSprop']
model = Sequential()
model.add(LSTM(144, batch_input_shape=(32, 1, 5), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer=_optimiser[0])
model.summary()
model.fit(train_x,train_y, batch_size=32,epochs=5)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (32, 144)                 86400     
_________________________________________________________________
dense_1 (Dense)              (32, 1)                   145       
Total params: 86,545
Trainable params: 86,545
Non-trainable params: 0
_________________________________________________________________

Epoch 1/5
730400/730400 [==============================] - 67s 91us/step - loss: 907.3272
Epoch 2/5
730400/730400 [==============================] - 66s 91us/step - loss: 830.0942
Epoch 3/5
730400/730400 [==============================] - 69s 94us/step - loss: 830.0792
Epoch 4/5
730400/730400 [==============================] - 67s 92us/step - loss: 830.0953
Epoch 5/5
730400/730400 [==============================] - 62s 85us/step - loss: 830.0832


In [11]:
test_x

array([[[   7,   10,    1,    5, 2013]],

       [[   7,   40,    5,    4, 2014]],

       [[   8,   13,    1,    3, 2014]],

       ...,

       [[   3,   29,    8,    5, 2016]],

       [[   6,   44,    1,    3, 2013]],

       [[   2,   46,   12,    2, 2016]]])

In [7]:
# univariate mlp example
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
# define dataset
X = array([[10, 20, 30], [20, 30, 40], [30, 40, 50], [40, 50, 60]])
y = array([40, 50, 60, 70])
# define model
model = Sequential()
model.add(Dense(100, activation='relu', input_dim=3))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=2000, verbose=0)
# demonstrate prediction
x_input = array([50, 60, 70])
x_input = x_input.reshape((1, 3))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[80.10915]]


In [10]:
print(X)
print(y)

[[10 20 30]
 [20 30 40]
 [30 40 50]
 [40 50 60]]
[40 50 60 70]
